In [144]:
import pandas as pd

data = pd.read_csv("FINALE.csv")
print(data.columns)

Index(['ETER_ID_year', 'ETER_ID', 'institution_name',
       'english_institution_name', 'reference_year', 'country_code',
       'name_of_the_city', 'latitude', 'longitude',
       'personnel_expenditure(PPP)', 'non-personnel_expenditure(PPP)',
       'expenditure_unclassified(PPP)', 'total_current_expenditure(PPP)',
       'capital_expenditure_(PPP)', 'total_core_budget(PPP)',
       'total_third_party_funding(PPP)', 'student_fees_funding(PPP)',
       'revenue_unclassified(PPP)', 'total_current_revenues(PPP)',
       'total_academic_staff(FTE)', 'total_staff_(FTE)',
       'lowest_degree_delivered', 'highest_degree_delivered',
       'total_students_enrolled_at_ISCED_5',
       'total_students_enrolled_at_ISCED_6',
       'total_students_enrolled_at_ISCED_7',
       'total_students_enrolled_ISCED_7_longdegree',
       'total_students_enrolled_ISCED_5-7', 'total_graduates_at_ISCED_5',
       'total_graduates_at_ISCED_6', 'total_graduates_at_ISCED_7',
       'total_graduates_at_ISCED_

In [145]:
new_data = data.filter([
         
       'total_academic_staff(FTE)', 'total_staff_(FTE)',
       'total_students_enrolled_ISCED_5-7', 'total_graduates_ISCED_5-7',
       'erasmus_total_incoming_students', 'erasmus_total_outgoing_students',
        ] , axis=1)
print(new_data.head(2))

  total_academic_staff(FTE) total_staff_(FTE)  \
0                         m                 m   
1                         m                 m   

  total_students_enrolled_ISCED_5-7 total_graduates_ISCED_5-7  \
0                             26602                         m   
1                              2915                         m   

  erasmus_total_incoming_students erasmus_total_outgoing_students  
0                               m                               m  
1                               m                               m  


In [146]:
newdata = new_data.apply(lambda x: x.str.replace(',','.'))
newdata = newdata.replace(["m" , "#VALUE!" , "xr" , "c" , "a" , "xc" , "x" , "nc" , "s" , "NaN"] , 0)
print(newdata.head(2))

  total_academic_staff(FTE) total_staff_(FTE)  \
0                         0                 0   
1                         0                 0   

  total_students_enrolled_ISCED_5-7 total_graduates_ISCED_5-7  \
0                             26602                         0   
1                              2915                         0   

  erasmus_total_incoming_students erasmus_total_outgoing_students  
0                               0                               0  
1                               0                               0  


In [147]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_of_data = pca.fit_transform(newdata)
print(pca_of_data)

[[19355.33348633  2988.68128218]
 [-4257.95254201  1326.7963076 ]
 [-7163.8810586   1122.27928557]
 ...
 [-6125.45761185   889.7739883 ]
 [-5798.49255403   815.66816431]
 [-6141.47407462   499.44225772]]


In [148]:
pca_of_data.min(0)

array([ -7163.8810586 , -19864.01575734])

In [149]:
pca_of_data.max(0)

array([3016396.82635921,   84214.82143013])

In [150]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=2, init="k-means++" )
y = kmeans.fit_transform(pca_of_data)

In [151]:
y.argmax(0)

array([2913, 2913])

In [152]:
pca_to_dataframe = pd.DataFrame(y , columns=["x" , "y"])
labels_todatafra = pd.DataFrame(kmeans.labels_ , columns=["label"])

In [153]:
pca_to_dataframe.index

RangeIndex(start=0, stop=17652, step=1)

In [154]:
final1 = pd.concat([pca_to_dataframe, labels_todatafra], axis=1 , ignore_index=False )

In [155]:
final1.head(10)

,x,y,label
0,23416.130699,22502.137677,1
1,900.575815,45385.906844,0
2,3298.186208,48254.618742,0
3,3298.186208,48254.618742,0
4,3298.186208,48254.618742,0
5,2310.605130,47230.509466,0
6,55505.269184,13780.728852,1
7,22585.248172,23279.237823,0
8,3298.186208,48254.618742,0
9,2624.360333,47560.318731,0


In [156]:
toconcat = data.filter(["institution_name" , "ETER_ID" , "reference_year", "country_code"])
final = pd.concat([final1, toconcat], axis=1 , ignore_index=False )

In [157]:
final.head(1)

,x,y,label,institution_name,ETER_ID,reference_year,country_code
0,23416.130699,22502.137677,1,Université Nice - Sophia-Antipolis,FR0001,2017,FR


In [158]:
data1 = pd.read_csv("statistic_per_uni.csv")
data1 = data1.filter(["missing_perc","cons_perc"])

In [159]:
final3 = pd.concat([final , data1], axis=1 , ignore_index=False )
final3.to_csv("fusion_kmeans.csv" , index=False)

In [160]:
final3.head(1)

,x,y,label,institution_name,ETER_ID,reference_year,country_code,missing_perc,cons_perc
0,23416.130699,22502.137677,1,Université Nice - Sophia-Antipolis,FR0001,2017,FR,0.6,0.2


In [161]:
y.max(0)


array([3021484.08942921, 2976840.70943953])

In [162]:
y.min(0)

array([  7.62934202, 480.19162331])

In [163]:
pca_to_dataframe2 = pd.DataFrame(pca_of_data , columns=["x" , "y"])

In [164]:
pca_final = pd.concat([pca_to_dataframe2, toconcat], axis=1 , ignore_index=False )
pca_final2 = pd.concat([pca_final, data1], axis=1 , ignore_index=False )

In [165]:
pca_final2.to_csv("pca.csv" , index=False)